In [122]:
# import dependencies 
import math 
import numpy as np
from itertools import combinations

In [134]:
def calculate_proportion(project, project_subset, funding_score):
    """
    Calculate the proportion of the funding score of a project relative to the total funding scores of all projects in the subset.

    Args:
    - project: The project for which the proportion is being calculated.
    - project_subset: The subset of projects among which the proportion is being calculated.
    - funding_score: Dictionary containing funding scores of all projects.

    Returns:
    - Proportion of the funding score of the project relative to the total funding scores of all projects in the subset.
    """
    total_subset_score = sum(funding_score[q] for q in project_subset)
    project_score = funding_score[project]
    proportion = project_score / total_subset_score
    
    return proportion

In [137]:
projects = ['0', '1']
fundingScore = {'0': 2, '1': 3}
min_funds = {'0': 30000, '1': 70000}
max_funds = {'0': 30000, '1': 70000}
total_budget = 100000

In [139]:
 # Set initial values for funding and excess
funding = {p: total_budget * calculate_proportion(p, projects, fundingScore) for p in projects}
print(funding)

{'0': 40000.0, '1': 60000.0}


In [136]:
def scale_funding(projects, fundingScore, mins, maxs, total_budget):
  '''
  input:
    projects - list (int) of projects
    fundingScore - dictionary indexed by projects. fundingScore[p] = output of COCM for project p
    mins - dictionary indexed by projects. mins[p] = minimum viable funding for project p
    maxs - dictionary indexed by projects. mins[p] = maximum viable funding for project p
    total budget - float. total amount of funding to give out.

  output:
    funding - dictionary indexed by projects. funding[p] = actual amount of funding to give to project p. funding[p] is either 0, or in between min[p] and max[p]
    excess - float. amount of unused funding (greater than 0 if, for example, total_budget is bigger than the sum of all maximum budgets)
  '''

  # subroutine: given a project and a subset of projects, what is fundingScore[p] as a percentage of the scores among projects in that set?
  proportion = lambda p,x : fundingScore[p] / sum(fundingScore[q] for q in x)
  
  for p in projects:
            print(f"Proportion for project {p}: {proportion(p, projects)}")

  # set initial values for funding and excess
  funding = {p: total_budget * proportion(p, projects) for p in projects}
   
  print(f"Project funding: {funding}")
  
  excess = 0

  # as we move funding around, we'll use these sets to keep track of how much funding different projects have
  above_max, at_max, between, below_min, zero = set(), set(), set(), set(), set()

  # a subroutine to make sure a project is in the appropriate set
  def change_set(p):
    for s in above_max, at_max, between, below_min, zero:
      s.discard(p)
    if   funding[p] > maxs[p] : above_max.add(p)
    elif funding[p] == maxs[p]: at_max.add(p)
    elif funding[p] >= mins[p]: between.add(p)
    elif funding[p] > 0       : below_min.add(p)
    else                      : zero.add(p)
    assert funding[p] >= 0

  # run the above subroutine project to put every project in the right set
  for p in projects: change_set(p)

  # a subroutine to take the current excess and redistribute it among projects in the set s
  def distribute_excess(s):
    for p in s:
      funding[p] += excess * proportion(p, s)
      change_set(p)

  # a subroutine to find the set of worst-performing projects with non-zero funding
  min_funding = lambda: {p for p in below_min if funding[p] / mins[p] == min(funding[q] / mins[q] for q in below_min)}

  # main loop
  while len(above_max) + len(below_min) > 0:

    # if any projects are above their maximum, remove the excess and redistribute it
    while len(above_max) > 0:
      for p in above_max.copy():
        # iterate through a copy because changing a set's size while you're iterating through it is not allowed
        excess += funding[p] - maxs[p]
        funding[p] = maxs[p]
        change_set(p)

      if len(below_min.union(between)) > 0:
        distribute_excess(below_min.union(between).copy())
        excess = 0

    # if there is still at least one project below its minimum level of funding, remove the worst performing one and redistribute its funding.
    if len(below_min) > 0:

      for p in min_funding().copy():
        excess += funding[p]
        funding[p] = 0
        change_set(p)

      if len(below_min.union(between)) > 0:
        distribute_excess(below_min.union(between).copy())
        excess = 0

    # after each pass in this while loop, len(above_max) is 0 and len(below_min) is at least 1 smaller than it was before.
    # So we are guaranteed to terminate.

  return funding, excess

In [135]:
projects = ['0', '1']
fundingScore = {'0': 2, '1': 3}
min_funds = {'0': 30000, '1': 70000}
max_funds = {'0': 30000, '1': 70000}
total_budget = 100000

print(fundingScore)
print(min_funds)
print(max_funds)

f, e = scale_funding(projects, fundingScore, min_funds, max_funds, total_budget)
print(f)
print(e)
print(sum(f[p] for p in projects) + e)

{'0': 2, '1': 3}
{'0': 30000, '1': 70000}
{'0': 30000, '1': 70000}
Proportion for project 0: 0.4
Proportion for project 1: 0.6
Project funding:{'0': 40000.0, '1': 60000.0}
{'0': 30000, '1': 70000.0}
0
100000.0
